In [ ]:
# ----------- 配置项 -------------
import pandas as pd
import glob
import matplotlib.pyplot as plt
import seaborn as sns
import os
import numpy as np
from matplotlib.ticker import MultipleLocator
from collections import defaultdict

# --- 全局配置 (在多次运行中保持不变) ---

plt.rcParams.update({'figure.max_open_warning': 0})

FONT_SIZES = {
    "ylabel": 16, "xlabel": 16, "xtick": 16,
    "ytick": 16, "legend": 16, "title": 16
}

try:
    plt.rcParams['font.family'] = 'Arial'
    print("✅ Successfully set global font to Arial.")
except RuntimeError:
    print("❌ ERROR: Arial font not found. Please ensure it is installed on your system.")
    print("ℹ️ Falling back to the default sans-serif font.")
    plt.rcParams['font.family'] = 'sans-serif'

# P值文件的根目录 (固定)
P_VALUE_BASE_DIR = "/data114_4/chenjy/DLBCL/model/group_evaluation"
print(f"✅ P-value base directory set to: {P_VALUE_BASE_DIR}")

# 指标名称映射 (固定)
METRIC_NAME_MAP_FOR_PVAL = {"AUROC": "ROC-AUC", "AUPRC": "PR-AUC"}
print(f"✅ P-value metric name mapping enabled: {METRIC_NAME_MAP_FOR_PVAL}")

# --- 辅助函数 ---

def format_p_value_text(p_val):
    """格式化p值为斜体的 'P = xxx' 字符串, 并处理nan值。"""
    if pd.isna(p_val):
        return ''
    if p_val < 0.001:
        return '$\\it{P}$ < 0.001'
    return f'$\\it{{P}}$ = {p_val:.3f}'

def load_center_data(root, center_name, metrics_list):
    """从指定路径加载并处理单个中心的数据。"""
    all_csvs = glob.glob(os.path.join(root, "metrics_by_*[0-9].csv"))
    grouped_files = defaultdict(list)
    for file in all_csvs:
        base = os.path.basename(file)
        group_key = base.split("_fold")[0] if "_fold" in base else "_".join(base.split("_")[:-1])
        grouped_files[group_key].append(file)
    
    all_means = []
    for group_key, file_list in grouped_files.items():
        if not file_list: continue
        try:
            df_all = [pd.read_csv(f).assign(fold=os.path.basename(f)) for f in sorted(file_list)]
            df = pd.concat(df_all, ignore_index=True)
            sample_df = pd.read_csv(file_list[0])
            metric_cols = ["Target", "N"] + metrics_list
            stratify_col_candidates = [col for col in sample_df.columns if col not in metric_cols]
            
            if not stratify_col_candidates:
                print(f"Warning: Could not determine stratify column for group {group_key}. Skipping.")
                continue
                
            stratify_col = stratify_col_candidates[0]
            df = df[~df[stratify_col].astype(str).str.lower().isin(["unknown", "not-known"])]
            
            if stratify_col.lower() == 'coo':
                cleaned_col = df[stratify_col].astype(str).str.strip().str.lower()
                df[stratify_col] = cleaned_col.replace({'gcb': 'GCB', 'nongcb': 'Non-GCB'})
            else:
                df[stratify_col] = df[stratify_col].astype(str).str.strip().str.capitalize()
            
            grouped = df.groupby([stratify_col, "Target"])
            agg_df = grouped.agg({**{metric: 'mean' for metric in metrics_list}, "N": "first"}).reset_index()
            stds = grouped[metrics_list].std().reset_index()
            for metric in metrics_list:
                agg_df[f"{metric}_std"] = stds[metric]
            
            agg_df["Stratify"] = agg_df[stratify_col]
            agg_df["Stratify_Col_Name"] = stratify_col
            agg_df["Center"] = center_name
            agg_df["Group"] = group_key
            all_means.append(agg_df)
        except Exception as e:
            print(f"❌ ERROR processing group {group_key} in {root}: {e}")

    if not all_means: return pd.DataFrame()
    return pd.concat(all_means, ignore_index=True)

# ----------- ✅ 主绘图函数 -------------
def generate_plots(root_beijing, root_tch, root_sxh, output_dir):
    """
    接收输入和输出路径, 加载所有数据并生成所有分组的图表。
    """
    print(f"\nProcessing data for output directory: {output_dir}")
    
    # 动态生成前缀映射
    center_to_prefix = {
        "PUTH": os.path.basename(root_beijing),
        "CHCAMS": os.path.basename(root_tch),
        "SXCH": os.path.basename(root_sxh),
    }
    print(f"✅ Center to prefix mapping for this run: {center_to_prefix}")

    metrics = ["Accuracy", "F1", "Precision", "Recall", "AUROC", "AUPRC"]
    fixed_colors = [(155/255, 206/255, 182/255), (235/255, 176/255, 145/255)]
    global_group_gap = 3

    # 加载数据
    print("Loading data...")
    df_beijing = load_center_data(root_beijing, "PUTH", metrics)
    df_tch     = load_center_data(root_tch, "CHCAMS", metrics)
    df_sxh     = load_center_data(root_sxh, "SXCH", metrics)
    merged = pd.concat([df_beijing, df_tch, df_sxh], ignore_index=True)
    
    if merged.empty:
        print("❌ No data loaded for this run. Skipping plotting.")
        return

    # 循环处理每个分组 (Age, COO, etc.)
    for group_key in merged["Group"].unique():
        print(f"\n🔹 Processing Group: {group_key}")
        df_group = merged[merged["Group"] == group_key].copy()
        if df_group.empty: continue
        
        group_output_dir = os.path.join(output_dir, group_key)
        os.makedirs(group_output_dir, exist_ok=True)
        stratify_title = df_group["Stratify_Col_Name"].iloc[0]

        # 准备p值查找路径所需的名称
        strat_for_path = stratify_title.lower()
        if strat_for_path == "age_bin": strat_for_path = "age"
        if strat_for_path == "localization": strat_for_path = "location"
        if strat_for_path == "coo": strat_for_path = "COO"
        
        # 准备p值CSV文件名所需的名称
        strat_for_filename = stratify_title
        if stratify_title.lower() == "age_bin": strat_for_filename = "Age"
        if stratify_title.lower() == "localization": strat_for_filename = "Location"
        if stratify_title.lower() == "coo": strat_for_filename = "COO"
        
        # 加载p值数据
        p_values_data = {}
        print("   -> Searching for p-value files...")
        for center_name_in_plot in df_group["Center"].unique():
            prefix = center_to_prefix.get(center_name_in_plot)
            if not prefix: continue
            
            p_value_folder = os.path.join(P_VALUE_BASE_DIR, f"{prefix}_{strat_for_path}_full_test")
            p_value_csv_file = os.path.join(p_value_folder, f"{strat_for_filename}Full_metrics_paired_test.csv")
            
            if os.path.exists(p_value_csv_file):
                try:
                    df_p = pd.read_csv(p_value_csv_file)
                    for _, row in df_p.iterrows():
                        p_values_data[(center_name_in_plot, row['Gene'], row['Metric'])] = row['p_t']
                    print(f"      ✅ Loaded {len(df_p)} p-values from: {p_value_csv_file}")
                except Exception as e:
                    print(f"      ❌ ERROR reading p-value file {p_value_csv_file}: {e}")
            else:
                print(f"      ⚠️ WARNING: P-value file not found at: {p_value_csv_file}")
        
        # 准备绘图数据
        df_plot = df_group[df_group["Target"] != "Overall"].copy()
        targets = sorted(df_plot["Target"].unique())
        stratify_vals = sorted(df_plot["Stratify"].unique(), key=lambda x: str(x))
        centers = sorted(df_plot["Center"].unique())
        color_map = {val: fixed_colors[i % len(fixed_colors)] for i, val in enumerate(stratify_vals)}
        n_stratify = len(stratify_vals)
        bar_width = 2.3 / n_stratify

        x_positions, x_labels, x_ticks_map = [], [], {}
        current_pos = 0
        for center in centers:
            for target in targets:
                label = f"$\\it{{{target}}}$\n{center}"
                x_labels.append(label)
                x_ticks_map[label] = current_pos
                x_positions.append(current_pos)
                current_pos += global_group_gap
            current_pos += 1.2

        # 循环绘制每个指标的图
        for metric in metrics:
            print(f"   ➤ Plotting metric: '{metric}'")
            fig, ax = plt.subplots(figsize=(17, 7))
            max_content_y = 1.0 

            for i, strat in enumerate(stratify_vals):
                offset = (i - n_stratify / 2 + 0.5) * bar_width
                bar_positions, values, errors = [], [], []
                sub_df_strat = df_plot[df_plot["Stratify"] == strat]

                for label in x_labels:
                    target_raw, center_text = label.split("\n")[0].replace("$\\it{", "").replace("}$", ""), label.split("\n")[1]
                    pos = x_ticks_map[label]
                    row = sub_df_strat[(sub_df_strat["Center"] == center_text) & (sub_df_strat["Target"] == target_raw)]
                    bar_positions.append(pos + offset)

                    if not row.empty:
                        mean_val = row[metric].iloc[0]
                        std_val = row[f"{metric}_std"].fillna(0).iloc[0]
                        values.append(mean_val)
                        errors.append([[mean_val - max(0, mean_val - std_val)], [min(1, mean_val + std_val) - mean_val]])
                            
                    else:
                        values.append(0); errors.append([[0], [0]])
                
                # 在这里，label参数是为了让legend知道画什么，但我们后面会删除legend，所以这里的label其实不再必要
                ax.bar(bar_positions, values, width=bar_width, yerr=np.array(errors).squeeze().T, capsize=4, label=strat, color=color_map[strat])

            for label in x_labels:
                target_raw, center_text = label.split("\n")[0].replace("$\\it{", "").replace("}$", ""), label.split("\n")[1]
                metric_for_pval = METRIC_NAME_MAP_FOR_PVAL.get(metric, metric)
                p_val = p_values_data.get((center_text, target_raw, metric_for_pval))
                p_text = format_p_value_text(p_val)

                if p_text:
                    sub_df_target = df_plot[(df_plot["Center"] == center_text) & (df_plot["Target"] == target_raw)]
                    if not sub_df_target.empty:
                        bar_top_y = min((sub_df_target[metric] + sub_df_target[f"{metric}_std"].fillna(0)).max(), 1.0)
                        text_y_pos = bar_top_y + 0.03
                        ax.text(x_ticks_map[label], text_y_pos, p_text, ha='center', va='bottom', fontsize=12, color='black')
                        max_content_y = max(max_content_y, text_y_pos)
            
            # =====================================================================
            # ✅ MODIFIED: 删除图例和上、右边框
            # =====================================================================
            ax.spines['top'].set_visible(False)
            ax.spines['right'].set_visible(False)
            # ax.legend(...) # 此行已被删除或注释
            # =====================================================================

            ax.set_ylabel(metric, fontsize=FONT_SIZES["ylabel"])
            ax.tick_params(axis="y", labelsize=FONT_SIZES["ytick"])
            ax.set_ylim(0, max(1.05, max_content_y + 0.05))
            ax.yaxis.set_major_locator(MultipleLocator(0.1))
            ax.set_xticks(x_positions)
            ax.set_xticklabels(x_labels, rotation=0, ha="center", fontsize=FONT_SIZES["xtick"])
            ax.set_xlabel("Target & Center Cohort", fontsize=FONT_SIZES["xlabel"], labelpad=10)
            
            out_path = os.path.join(group_output_dir, f"{metric}_barplot_with_pval_text_no_box_no_legend.pdf")
            plt.savefig(out_path, dpi=300, bbox_inches='tight')
            # plt.show()
            plt.close(fig)
            print(f"      ✅ Saved plot to: {out_path}")

# ----------- ✅ 主执行模块 -------------
if __name__ == "__main__":
    run_configs = [
        {
            "beijing": "/data114_4/chenjy/DLBCL/model/group_csv_beijing/BH",
            "tch": "/data114_4/chenjy/DLBCL/model/group_csv_beijing/TCH",
            "sxh": "/data114_4/chenjy/DLBCL/model/group_csv_beijing/SXH",
            "output": "/data114_4/chenjy/DLBCL/model/SI-MIL/group_vis/beijing_tch/"
        }
    ]

    # 循环执行所有配置
    for i, config in enumerate(run_configs):
        print(f"\n\n{'='*80}")
        print(f"🚀 STARTING RUN {i+1}/{len(run_configs)}")
        print(f"{'='*80}")
        
        # 确保输出目录存在
        os.makedirs(config["output"], exist_ok=True)
        
        generate_plots(
            root_beijing=config["beijing"],
            root_tch=config["tch"],
            root_sxh=config["sxh"],
            output_dir=config["output"]
        )
    
    print(f"\n\n{'='*80}")
    print("✅ All plotting runs completed successfully!")
    print(f"{'='*80}")